<a href="https://colab.research.google.com/github/olyandrevn/vk_project/blob/main/notebooks/experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
data = '/content/drive/MyDrive/мл-задача/'
train_data = data + 'train_df.csv'
test_data = data + 'test_df.csv'

## Data Exploration



In [3]:
import pandas as pd

df_train = pd.read_csv(train_data)
df_test = pd.read_csv(test_data)

feature_0 -- all values 9 \\
feature_1 -- {0, 1, 2, 3} \\
feature_2 -- {0, 1, 2, 3, 4, 5, 6} \\
feature_3 == feature_4 \\
feature_5 -- {0, 1, 2, 3, 4, 5, 6} \\

...

feature_73 == feature_74 == feature_75 == {0}


In [4]:
df_train.target.value_counts()

0    14759
1      322
Name: target, dtype: int64

In [5]:
for col in df_train.columns:
    print(f'{df_train[col].value_counts()}\n')
    print('----------------------------\n')

156182    20
8591      20
226704    20
227432    20
315998    20
          ..
155433     1
271166     1
178343     1
387764     1
303366     1
Name: search_id, Length: 1000, dtype: int64

----------------------------

9    15081
Name: feature_0, dtype: int64

----------------------------

0    13592
2      733
1      406
3      350
Name: feature_1, dtype: int64

----------------------------

0    12544
1      638
2      587
4      567
3      418
5      294
6       33
Name: feature_2, dtype: int64

----------------------------

1    9499
0    5582
Name: feature_3, dtype: int64

----------------------------

20    9499
9     5582
Name: feature_4, dtype: int64

----------------------------

4    13708
3     1083
2      154
0       95
1       35
6        6
Name: feature_5, dtype: int64

----------------------------

25    2117
46    2053
31    1324
26     902
49     856
40     854
34     799
0      684
36     672
38     647
43     601
30     465
28     384
33     349
45     347
47     327


Был проведен анализ корреляции на проверку зависимости разных фичей; очевидно зависимы(даже одинаковы с точностью до значений) оказались `feature_3` и `feature_4`.

Также в таблице корреляций присутствуют `NaN` значения у `feature_0`, `feature_73`, `feature_74` и `feature_75` -- это фичи с единственным значением -- очевидно не информативны.

In [6]:
import pandas as pd
import numpy as np
from collections import Counter

corr = df_train.corr()
high_corr_pairs = set()
high_corr_cols = Counter({col : 0 for col in corr.columns})

for row in corr.index:
    for col in corr.columns:
        score = corr.loc[row, col]
        if score >= 0.89 and row != col:
            if (col, row, score) not in high_corr_pairs:
                high_corr_pairs.add((row, col, score))
            high_corr_cols[col] += 1

print('\n'.join([''.join(str(pair)) for pair in high_corr_pairs]))
print(high_corr_cols)

('feature_71', 'feature_72', 0.9002601212018557)
('feature_77', 'feature_78', 0.9679486235821477)
('feature_53', 'feature_63', 0.9257486924831281)
('feature_12', 'feature_60', 0.9077267427515578)
('feature_3', 'feature_4', 0.9999999999999989)
('feature_63', 'feature_65', 0.9378349104319865)
Counter({'feature_63': 2, 'feature_3': 1, 'feature_4': 1, 'feature_12': 1, 'feature_53': 1, 'feature_60': 1, 'feature_65': 1, 'feature_71': 1, 'feature_72': 1, 'feature_77': 1, 'feature_78': 1, 'search_id': 0, 'feature_0': 0, 'feature_1': 0, 'feature_2': 0, 'feature_5': 0, 'feature_6': 0, 'feature_7': 0, 'feature_8': 0, 'feature_9': 0, 'feature_10': 0, 'feature_11': 0, 'feature_13': 0, 'feature_14': 0, 'feature_15': 0, 'feature_16': 0, 'feature_17': 0, 'feature_18': 0, 'feature_19': 0, 'feature_20': 0, 'feature_21': 0, 'feature_22': 0, 'feature_23': 0, 'feature_24': 0, 'feature_25': 0, 'feature_26': 0, 'feature_27': 0, 'feature_28': 0, 'feature_29': 0, 'feature_30': 0, 'feature_31': 0, 'feature_32':

In [7]:
corr

,search_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,target
search_id,1.000000,NaN,-0.005195,-0.035058,0.034465,0.034465,-0.023497,-0.008107,-0.027329,-0.005243,...,0.012933,-0.006893,0.000358,NaN,NaN,NaN,0.012530,0.036139,0.023331,-0.005972
feature_0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
feature_1,-0.005195,NaN,1.000000,-0.017054,-0.054672,-0.054672,0.038106,0.045054,0.514021,0.222314,...,0.023376,0.076516,0.101744,NaN,NaN,NaN,0.022636,-0.025528,-0.023964,-0.000285
feature_2,-0.035058,NaN,-0.017054,1.000000,0.066014,0.066014,-0.008419,-0.024244,-0.051710,-0.048600,...,-0.026022,0.000258,-0.023006,NaN,NaN,NaN,-0.028082,-0.005987,-0.001722,-0.008337
feature_3,0.034465,NaN,-0.054672,0.066014,1.000000,1.000000,-0.007873,0.017213,-0.057525,0.005045,...,-0.050584,0.142552,0.077396,NaN,NaN,NaN,0.044092,0.022348,0.023254,-0.037827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
feature_75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
feature_76,0.012530,NaN,0.022636,-0.028082,0.044092,0.044092,0.029528,0.126931,0.050992,-0.012849,...,0.382821,0.433373,0.471807,NaN,NaN,NaN,1.000000,0.061974,0.076301,0.043589
feature_77,0.036139,NaN,-0.025528,-0.005987,0.022348,0.022348,-0.051923,-0.010225,-0.036952,0.002591,...,0.083546,0.017872,0.035760,NaN,NaN,NaN,0.061974,1.000000,0.967949,0.005084
feature_78,0.023331,NaN,-0.023964,-0.001722,0.023254,0.023254,-0.046336,-0.027883,-0.030444,0.004053,...,0.076135,0.041132,0.053233,NaN,NaN,NaN,0.076301,0.967949,1.000000,0.005447


## Data Preparation

In [ ]:
!pip install catboost

Были попытки удалять/оставлять различные наборы фичей; удаление сильно коррелирущих фичей не улучшало метрику NDCG.

Эмпирическим методом было выяснено, что на данном наборе фичей получается наибольшее значение метрики NDCG. В частности, замечу, что `feature_24`, `feature_55`, `feature_77`, `feature_78` играют важную роль, тк при их отсутствии качество значителько падает. Однако каких-то серьезных доказательств почему оно так у меня нет, возможно необходим какой-то хитрый анализ.

```
leave_features = features[5:25] + features[51:73] + features[77:78]
```

In [12]:
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit
from catboost import CatBoostClassifier, CatBoostRanker, Pool

gss = GroupShuffleSplit(n_splits=1, train_size=0.8, random_state=42)

for train_idx, val_idx in gss.split(df_train, groups=df_train['search_id']):
    df_train_split = df_train.iloc[train_idx]
    df_val_split = df_train.iloc[val_idx]

features = [f'feature_{i}' for i in range(79)]

leave_features = features[5:25] + features[51:73] + features[77:78]

X_train = df_train_split[leave_features]
y_train = df_train_split['target']
queries_train = df_train_split['search_id']

X_val = df_val_split[leave_features]
y_val = df_val_split['target']
queries_val = df_val_split['search_id']

X_test = df_test[leave_features]
y_test = df_test['target']
queries_test = df_test['search_id']

In [ ]:
train = Pool(
    data=X_train,
    label=y_train,
    group_id=queries_train
)

val = Pool(
    data=X_val,
    label=y_val,
    group_id=queries_val
)

test = Pool(
    data=X_test,
    label=y_test,
    group_id=queries_test
)

## Model Training: CatboostRanker


In [ ]:
default_parameters = {
    'iterations': 1000,
    'verbose': False,
    'random_seed': 0,
}

parameters = {}

In [ ]:
from copy import deepcopy

def fit_model(loss_function, additional_params=None, train_pool=train, val_pool=val):
    parameters = deepcopy(default_parameters)
    parameters['loss_function'] = loss_function
    parameters['train_dir'] = loss_function

    if additional_params is not None:
        parameters.update(additional_params)

    model = CatBoostRanker(**parameters)
    model.fit(train_pool, eval_set=val_pool, plot=True)

    return model


## Model Evaluation


In [ ]:
loss_functions = ['RMSE', 'QueryRMSE', 'PairLogit', 'YetiRank']

for loss_function in loss_functions:
    print(f'{loss_function}:')
    model = fit_model(loss_function)

    ndcg = model.score(test)
    print(f"NDCG value: {ndcg}")

RMSE:


/usr/local/lib/python3.10/dist-packages/catboost/core.py:6325: RuntimeWarning: Regression loss ('RMSE') ignores an important ranking parameter 'group_id'
  warnings.warn("Regression loss ('{}') ignores an important ranking parameter 'group_id'".format(loss_function), RuntimeWarning)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

NDCG value: 0.9238016841043597
QueryRMSE:


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

NDCG value: 0.9257775324114967
PairLogit:


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

NDCG value: 0.9173770197562464
YetiRank:


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

NDCG value: 0.9362712791728599
